In [1]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession

# spark-sql --version
# setting environ variables essentially
# we're also installing? libraries straight by these commands rather than manually.
# https://hadoop.apache.org/docs/r2.8.0/hadoop-aws/tools/hadoop-aws/index.html#Changing_Authentication_Providers
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1') # basically most recent version as my spark version
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.access.key', os.environ.get('aws_access_key_id'))
conf.set('spark.hadoop.fs.s3a.secret.key', os.environ.get('aws_secret_access_key'))

# building it via a config rather than a raw .appName()
spark = SparkSession.builder.config(conf=conf).getOrCreate()

23/03/16 14:33:18 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
23/03/16 14:33:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/jacob/.local/share/virtualenvs/pyspark_prac-QhYfwHaC/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jacob/.ivy2/cache
The jars for the packages stored in: /home/jacob/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7d7728f8-f7ce-48a1-84fe-3fcbbfdbc95d;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.1 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.901 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 133ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-----------------------------------

23/03/16 14:33:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
## STEP 1 Read file from S3
df = spark.read.parquet('s3a://jyablonski-iceberg/reddit_comment_data-2023-03-06.parquet', inferSchema = True, header = True)
# df.printSchema()
# df2 = df.toPandas()
df.createTempView('nba_data')

23/03/16 14:33:20 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
## STEP 2 Transform the file and add or remove things
df5 = spark.sql("SELECT * FROM nba_data limit 5;")
df_spark = df5.collect() # will pull it into a python list

In [4]:
# STEP 3 Write it back to S3
s3_dest_path = "s3a://jacobsbucket97-dev/pyspark/nba_tweets.csv"

# writing to a test2/ prefix with _SUCCESS file and the parquet file inside of it
df.coalesce(1).write.parquet('s3a://jyablonski-iceberg/test2/')

In [11]:
df3 = spark.read.parquet('s3a://jacobsbucket97/pyspark/nba_tweets_v2.parquet/part-00000-f4044b57-7cdf-4d1c-8065-80d54fa346cf-c000.snappy.parquet')
df3_pandas = df3.toPandas()